In [1]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

# Load the dataset
dataset_name = "ENZYMES"
path = "./data/tmp/ENZYMES"
original_dataset = TUDataset(root=path, name=dataset_name, use_node_attr=True)

# Function to add equal edge weights to a graph
def add_edge_weights(data):
    edge_index = data.edge_index
    edge_weight = torch.ones(edge_index.shape[1], dtype=torch.float)
    return Data(edge_index=edge_index, x=data.x, y=data.y, edge_weight=edge_weight)

# Create a new dataset with added edge weights
dataset = [add_edge_weights(data) for data in original_dataset]

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [2]:
n_input = dataset[0].x.size(1)

In [3]:
from experiments.classifier import GraphClassifierTimeConv
from experiments.train import train, test
from torch.optim import Adam

# Set the device to use for training (either 'cpu' or 'cuda' if GPU is available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ts = torch.linspace(0, 1, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
model = GraphClassifierTimeConv(pde='heat', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 2.6021, Accuracy: 0.1625
Epoch 101/1000, Loss: 1.5910, Accuracy: 0.3729
Epoch 201/1000, Loss: 1.4056, Accuracy: 0.4417
Epoch 301/1000, Loss: 1.2514, Accuracy: 0.5167
Epoch 401/1000, Loss: 1.1314, Accuracy: 0.5854
Epoch 501/1000, Loss: 1.0771, Accuracy: 0.5958
Epoch 601/1000, Loss: 0.9988, Accuracy: 0.6354
Epoch 701/1000, Loss: 0.9656, Accuracy: 0.6458
Epoch 801/1000, Loss: 0.9325, Accuracy: 0.6646
Epoch 901/1000, Loss: 0.9079, Accuracy: 0.6896
Test accuracy: 44.17%


In [4]:

ts = torch.linspace(torch.pi/2, torch.pi, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
model = GraphClassifierTimeConv(pde='wave', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 6.7375, Accuracy: 0.1938
Epoch 101/1000, Loss: 1.5641, Accuracy: 0.3625
Epoch 201/1000, Loss: 1.3735, Accuracy: 0.4625
Epoch 301/1000, Loss: 1.2545, Accuracy: 0.5312
Epoch 401/1000, Loss: 1.1836, Accuracy: 0.5604
Epoch 501/1000, Loss: 1.1406, Accuracy: 0.5729
Epoch 601/1000, Loss: 1.0924, Accuracy: 0.6083
Epoch 701/1000, Loss: 1.0586, Accuracy: 0.6229
Epoch 801/1000, Loss: 1.0665, Accuracy: 0.5917
Epoch 901/1000, Loss: 0.9978, Accuracy: 0.6438
Test accuracy: 51.67%


In [6]:
from experiments.gcn import GCNClassifier
# Create your model, optimizer, and criterion
model = GCNClassifier(num_features=n_input, hidden_channels=16, num_classes=6)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, test_loader, device)

Epoch 1/1000, Loss: 4.5859, Accuracy: 0.1271
Epoch 101/1000, Loss: 1.6818, Accuracy: 0.2604
Epoch 201/1000, Loss: 1.6444, Accuracy: 0.2729
Epoch 301/1000, Loss: 1.6008, Accuracy: 0.3354
Epoch 401/1000, Loss: 1.5754, Accuracy: 0.3521
Epoch 501/1000, Loss: 1.5440, Accuracy: 0.3667
Epoch 601/1000, Loss: 1.5224, Accuracy: 0.3896
Epoch 701/1000, Loss: 1.5103, Accuracy: 0.3729
Epoch 801/1000, Loss: 1.4860, Accuracy: 0.3937
Epoch 901/1000, Loss: 1.4636, Accuracy: 0.4167
Test accuracy: 35.00%
